In [1]:
import os
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = 'calvinandpogs-ee148'

role = sagemaker.get_execution_role()
print(bucket, role)
print(os.getcwd())

calvinandpogs-ee148 arn:aws:iam::652516965730:role/service-role/AmazonSageMaker-ExecutionRole-20210513T011299
/home/ec2-user/SageMaker/atrw/models/yolov5


In [9]:
# TENSORFLOW CONTAINER (PY37)

from sagemaker.tensorflow import TensorFlow

# Store-true parameters not working

estimator = TensorFlow(entry_point='test.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 1920,
                            'task': 'test',
                            'weights': 'SM_CHANNEL_MODEL/best.pt',
                            'data': 'atrw.yaml',
                            'save-s3': False
                        }
)

In [15]:
estimator.fit({'model': f's3://{bucket}/models/yolov5/train-full/',
               'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
               'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
               'test': f's3://{bucket}/atrw/detection/test/'})

2021-05-26 07:29:36 Starting - Starting the training job...
2021-05-26 07:29:59 Starting - Launching requested ML instancesProfilerReport-1622014174: InProgress
......
2021-05-26 07:31:07 Starting - Preparing the instances for training.........
2021-05-26 07:32:21 Downloading - Downloading input data......
2021-05-26 07:33:34 Training - Downloading the training image...
2021-05-26 07:34:00 Training - Training image download completed. Training in progress.2021-05-26 07:33:54.426453: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-26 07:33:54.431932: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-05-26 07:33:54.568637: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-26 07:33:57,518 sagemaker-training-toolkit INFO     Importe